# Mini-Project 5.3 — Ship Engine Anomaly Detection (Resubmission)

**Author:** Michail Naoum
**Date:** 2025-10-19


In [ ]:
# 1)Imports & Settings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

from typing import Dict, Tuple
sns.set_context("notebook")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [ ]:
# 2)Data Load & EDA

URL = "https://raw.githubusercontent.com/fourthrevlxd/cam_dsb/main/engine.csv"
df = pd.read_csv(URL)

print(f"Rows: {df.shape[0]} | Columns: {df.shape[1]}")

eda_tbl = df.describe(percentiles=[0.95]).T.rename(columns={"50%":"median"})
eda_tbl = eda_tbl[["mean","median","95%","min","max"]]
display(eda_tbl)

miss = df.isna().sum()
dups = int(df.duplicated().sum())
print(f"Missing values per column (non-zeros only):\n{miss[miss>0].to_string() if (miss>0).any() else 'None'}")
print(f"Duplicate rows: {dups}")

for col in df.columns:
    fig, ax = plt.subplots(1,2, figsize=(9,2.8))
    sns.histplot(df[col], kde=True, ax=ax[0])
    ax[0].set_title(f"Hist — {col}")
    sns.boxplot(x=df[col], ax=ax[1])
    ax[1].set_title(f"Box — {col}")
    plt.tight_layout()
    plt.show()


In [ ]:
# 3)IQR Outliers (Per-Feature Flags + K-Sweep)

def iqr_bounds(s: pd.Series, k: float = 1.5) -> Tuple[float, float]:
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

# Explanation:
# - Each feature gets an IQR lower/upper bound; values outside become per-feature outliers (1).
# - `iqr_anomaly` marks a row anomaly if ≥ K per-feature outliers.
# - We sweep K ∈ {1,2,3,4} and select K that yields ~1–5% (target ≈3%).

df_iqr_flags = pd.DataFrame(index=df.index)
bounds: Dict[str, Tuple[float,float]] = {}
for col in df.columns:
    lo, hi = iqr_bounds(df[col], 1.5)
    bounds[col] = (lo, hi)
    df_iqr_flags[f"{col}_outlier"] = ((df[col] < lo) | (df[col] > hi)).astype(int)

thr_table = pd.DataFrame(bounds, index=["lower_bound","upper_bound"]).T
display(thr_table)

flag_sum = df_iqr_flags.sum(axis=1)
rows = []
for K in (1,2,3,4):
    mask = (flag_sum >= K).astype(int)
    rows.append({"K_features": K, "anomaly_count": int(mask.sum()), "anomaly_rate": mask.mean()})
iqr_k_table = pd.DataFrame(rows)
display(iqr_k_table)

target = 0.03
cands = iqr_k_table[(iqr_k_table["anomaly_rate"]>=0.01) & (iqr_k_table["anomaly_rate"]<=0.05)]
select_tbl = cands if not cands.empty else iqr_k_table
best_k = int(select_tbl.iloc[(select_tbl["anomaly_rate"] - target).abs().argmin()]["K_features"])

df["iqr_anomaly"] = (flag_sum >= best_k).astype(int)
print(f"Chosen K = {best_k} | IQR anomalies = {int(df['iqr_anomaly'].sum())} | rate = {df['iqr_anomaly'].mean():.2%}")


In [ ]:
# 4) Scaling & PCA (2D)

X_raw = df.to_numpy()
X_robust = RobustScaler().fit_transform(X_raw)
X_std = StandardScaler().fit_transform(X_raw)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_robust)
print("PCA explained variance ratio (2D):", pca.explained_variance_ratio_,
      "| cumulative:", pca.explained_variance_ratio_.sum())


In [ ]:
# 5)One-Class SVM (RBF) — Parameter Sweep

# 5) One-Class SVM (RBF) — Parameter Sweep  (FIXED)
from sklearn.svm import OneClassSVM

def sweep_ocsvm(X, nus=(0.01,0.02,0.03,0.05), gammas=("scale","auto",0.1,0.5,1.0)):
    rows, preds = [], {}
    for nu in nus:
        for g in gammas:
            m = OneClassSVM(kernel="rbf", nu=nu, gamma=g)
            m.fit(X)
            y = m.predict(X)  # -1 anomaly, 1 normal
            rows.append({
                "nu": nu,
                "gamma": g,
                "model": f"OCSVM(nu={nu},gamma={g})",
                "anomaly_count": int((y == -1).sum()),
                "anomaly_rate": (y == -1).mean()
            })
            preds[(nu, g)] = y
    tbl = pd.DataFrame(rows).sort_values("anomaly_rate").reset_index(drop=True)
    return tbl, preds

oc_tbl, oc_preds = sweep_ocsvm(X_std)
display(pd.concat([oc_tbl.head(6), oc_tbl.tail(6)]))

# pick closest to 3%
idx = (oc_tbl["anomaly_rate"] - 0.03).abs().argmin()
best_row = oc_tbl.iloc[idx]
best_nu = float(best_row["nu"])
best_gamma = best_row["gamma"]

# Primary lookup
key = (best_nu, best_gamma)
if key not in oc_preds:
    # Fallback to handle float-string mismatches or tiny float precision diffs
    def _match_key(k):
        nu_k, g_k = k
        nu_ok = abs(nu_k - best_nu) < 1e-12
        if isinstance(best_gamma, str) and isinstance(g_k, str):
            g_ok = (g_k == best_gamma)
        elif isinstance(best_gamma, str) != isinstance(g_k, str):
            g_ok = False
        else:
            g_ok = abs(float(g_k) - float(best_gamma)) < 1e-12
        return nu_ok and g_ok

    matches = [k for k in oc_preds.keys() if _match_key(k)]
    if matches:
        key = matches[0]
    else:
        raise KeyError(f"No matching key in oc_preds for nu={best_nu}, gamma={best_gamma}.")

ocsvm_labels = oc_preds[key]
best_oc_name = best_row["model"]
print(f"Chosen {best_oc_name} → anomalies={int((ocsvm_labels==-1).sum())} | rate={(ocsvm_labels==-1).mean():.2%}")



In [ ]:
# 6) Isolation Forest (Unscaled Primary) — Sweep

def sweep_if(X, contaminations=(0.01,0.02,0.03,0.05), n_estimators=(200,400)):
    rows, preds = [], {}
    for c in contaminations:
        for n in n_estimators:
            m = IsolationForest(contamination=c, n_estimators=n, random_state=42, n_jobs=-1)
            m.fit(X)
            y = m.predict(X)  # -1 anomaly, 1 normal
            rows.append({"model": f"IF(c={c},n={n})", "anomaly_count": int((y==-1).sum()), "anomaly_rate": (y==-1).mean()})
            preds[(c,n)] = y
    tbl = pd.DataFrame(rows).sort_values("anomaly_rate").reset_index(drop=True)
    return tbl, preds

if_unscaled_tbl, if_unscaled_preds = sweep_if(X_raw)     # primary
if_scaled_tbl,   if_scaled_preds   = sweep_if(X_std)     # comparison

display(pd.concat([if_unscaled_tbl.head(6), if_unscaled_tbl.tail(6)]).style.set_caption("IF (unscaled) — sweep"))
display(pd.concat([if_scaled_tbl.head(4), if_scaled_tbl.tail(4)]).style.set_caption("IF (scaled) — sweep"))

idx2 = (if_unscaled_tbl["anomaly_rate"] - 0.03).abs().argmin()
best_if = if_unscaled_tbl.iloc[idx2]
best_if_name = best_if["model"]
best_c = float(best_if_name.split('c=')[1].split(',')[0])
best_n = int(best_if_name.split('n=')[1].split(')')[0])
iforest_labels = if_unscaled_preds[(best_c, best_n)]
print(f"Chosen (unscaled) {best_if_name} → anomalies={int((iforest_labels==-1).sum())} | rate={(iforest_labels==-1).mean():.2%}")


In [ ]:
# 7) PCA 2D Visuals

import numpy as np

def plot_pca(X2d, mask, title):
    lbl = np.where(mask, "Anomaly", "Normal")
    palette = {"Normal":"tab:blue","Anomaly":"tab:red"}
    fig, ax = plt.subplots(figsize=(6.6,4.8))
    sns.scatterplot(x=X2d[:,0], y=X2d[:,1], hue=lbl, ax=ax, palette=palette, s=24, alpha=0.85)
    ax.set_title(title)
    ax.set_xlabel("PCA 1")
    ax.set_ylabel("PCA 2")
    plt.legend()
    plt.tight_layout()
    plt.show()

mask_iqr = df["iqr_anomaly"].astype(bool).values
mask_oc  = (ocsvm_labels == -1)
mask_if  = (iforest_labels == -1)

plot_pca(X_pca, mask_iqr, "PCA — IQR anomalies")
plot_pca(X_pca, mask_oc,  f"PCA — {best_oc_name}")
plot_pca(X_pca, mask_if,  f"PCA — {best_if_name} (unscaled)")


In [ ]:
# 8)Final Comparison & Reflections

def jaccard(a, b):
    inter = (a & b).sum(); uni = (a | b).sum()
    return inter/uni if uni else 0.0

comp_tbl = pd.DataFrame({
    "method": [f"IQR (K={best_k})", best_oc_name, f"{best_if_name} (unscaled)"],
    "anomaly_count": [int(df['iqr_anomaly'].sum()), int((ocsvm_labels==-1).sum()), int((iforest_labels==-1).sum())],
    "anomaly_rate": [df['iqr_anomaly'].mean(), (ocsvm_labels==-1).mean(), (iforest_labels==-1).mean()]
})
display(comp_tbl)

mask_iqr = df["iqr_anomaly"].astype(bool).values
mask_oc  = (ocsvm_labels == -1)
mask_if  = (iforest_labels == -1)

overlap_tbl = pd.DataFrame({
    "pair": ["IQR vs OCSVM", "IQR vs IF", "OCSVM vs IF"],
    "jaccard": [jaccard(mask_iqr,mask_oc), jaccard(mask_iqr,mask_if), jaccard(mask_oc,mask_if)],
    "intersections": [int((mask_iqr&mask_oc).sum()), int((mask_iqr&mask_if).sum()), int((mask_oc&mask_if).sum())]
})
display(overlap_tbl)


## Summary of Improvements (Resubmission)
- Added **IQR K-sweep (K=1..4)** and selected **K** targeting **~3%** anomalies.
- Included **IQR bounds table** and clear explanation of `iqr_anomaly`.
- Implemented **OCSVM parameter sweep** (ν, γ) with compact results view.
- Implemented **Isolation Forest (UNSCALED primary)** sweep (+ scaled comparison).
- Reported **PCA explained variance ratio** and provided final 2D plots.
- Standardised variable names and notebook section headings.
- Kept outputs minimal: final tables/plots only.
